In [21]:
import pandas as pd
import pandas_gbq
import tqdm

import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

# Project ID
project_id = "ccnr-success"

What are the leading research organizations pursuing any COVID-19 research? Who are the leading research organizations pursuing COVID-19 vaccine research?

In [51]:
sql = """
select 
  counts.id, 
  counts.pubcount,  
  orgs.name,
  orgs.address.country as country,
  orgs.address.latitude as latitude,
  orgs.address.longitude as longitude
from
(
  select 
    orgs.id,
    count(orgs.id) as pubcount
  from (
    SELECT id, orgid
    FROM `covid-19-dimensions-ai.data.publications`,
    UNNEST(research_orgs) orgid
    WHERE type = "article"  -- limit to research articles
    AND ('COVID-19' in UNNEST(concepts.concept) or "new coronavirus" in UNNEST(concepts.concept) or "SARS-CoV-2" in UNNEST(concepts.concept)
          OR 'ncov' in UNNEST(concepts.concept) or "novel coronavirus" in UNNEST(concepts.concept) or "2019-ncov" in UNNEST(concepts.concept)
          OR "severe acute respiratory syndrome coronavirus" in UNNEST(concepts.concept) or 'coronavirus disease 2019' in UNNEST(concepts.concept)
          OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  ) as pubs 
  left join `covid-19-dimensions-ai.data.grid` as orgs on pubs.orgid = orgs.id
  group by orgs.id
) as counts
left join `covid-19-dimensions-ai.data.grid` as orgs on counts.id = orgs.id
order by pubcount DESC

"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 776bead1-0dd4-4a35-94f8-27f019a0b2a7
Job ID: 776bead1-0dd4-4a35-94f8-27f019a0b2a7
Query done.
Cache hit.

Query done.
Cache hit.

Downloading: 100%|███████████████████| 22202/22202 [00:02<00:00, 10604.49rows/s]
Got 22202 rows.

Got 22202 rows.



,id,pubcount,name,country,latitude,longitude
0,grid.38142.3c,3298,Harvard University,United States,42.377052,-71.116653
1,grid.33199.31,1922,Huazhong University of Science and Technology,China,30.508183,114.414742
2,grid.17063.33,1869,University of Toronto,Canada,43.661667,-79.394997
3,grid.21107.35,1860,Johns Hopkins University,United States,39.328888,-76.620277
4,grid.4991.5,1824,University of Oxford,United Kingdom,51.753437,-1.254010
...,...,...,...,...,...,...
22197,grid.490170.b,1,Fuling Center Hospital of Chongqing,China,29.709438,107.395531
22198,grid.459318.2,1,Australian College of Applied Psychology,Australia,-33.865334,151.205322
22199,grid.483908.e,1,Shanghai Hospital Development Center,China,31.238739,121.455162
22200,grid.465956.d,1,LUCA School of Arts,Belgium,50.861362,4.367237


In [41]:
# Maybe show differences in rank between these two? Some sort of flow-like graph?
sql = """
select 
  counts.id, 
  counts.pubcount,  
  orgs.name,
  orgs.address.country as country,
  orgs.address.latitude as latitude,
  orgs.address.longitude as longitude
from
(
  select 
    orgs.id,
    count(orgs.id) as pubcount
  from (
    SELECT id, orgid
    FROM `covid-19-dimensions-ai.data.publications`,
    UNNEST(research_orgs) orgid
    where type = 'article'
    AND ('COVID-19' in UNNEST(concepts.concept) or "new coronavirus" in UNNEST(concepts.concept) or "SARS-CoV-2" in UNNEST(concepts.concept)
          OR 'ncov' in UNNEST(concepts.concept) or "novel coronavirus" in UNNEST(concepts.concept) or "2019-ncov" in UNNEST(concepts.concept)
          OR "severe acute respiratory syndrome coronavirus" in UNNEST(concepts.concept) or 'coronavirus disease 2019' in UNNEST(concepts.concept)
          OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  ) as pubs 
  left join `covid-19-dimensions-ai.data.grid` as orgs on pubs.orgid = orgs.id
  group by orgs.id
) as counts
left join `covid-19-dimensions-ai.data.grid` as orgs on counts.id = orgs.id
order by pubcount DESC

"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 58ca3738-d54e-486f-bf7b-7bb219983c5b
Job ID: 58ca3738-d54e-486f-bf7b-7bb219983c5b
Query done.
Processed: 731.0 MB Billed: 731.0 MB
Query done.
Processed: 731.0 MB Billed: 731.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Downloading: 100%|████████████████████| 22202/22202 [00:02<00:00, 8847.99rows/s]
Got 22202 rows.

Got 22202 rows.

Total time taken 12.06 s.
Finished at 2022-04-27 12:06:57.
Total time taken 12.06 s.
Finished at 2022-04-27 12:06:57.


,id,pubcount,name,country,latitude,longitude
0,grid.38142.3c,3298,Harvard University,United States,42.377052,-71.116653
1,grid.33199.31,1922,Huazhong University of Science and Technology,China,30.508183,114.414742
2,grid.17063.33,1869,University of Toronto,Canada,43.661667,-79.394997
3,grid.21107.35,1860,Johns Hopkins University,United States,39.328888,-76.620277
4,grid.4991.5,1824,University of Oxford,United Kingdom,51.753437,-1.254010
...,...,...,...,...,...,...
22197,grid.441677.7,1,Universidad Nacional José María Arguedas,Peru,-13.656001,-73.387886
22198,grid.263652.6,1,Sinclair Community College,United States,39.757004,-84.198471
22199,grid.454577.0,1,Granite State College,United States,43.192921,-71.526978
22200,grid.488859.0,1,L’Institut du Cancer Courlancy Reims,France,49.245121,4.018232


In [48]:
# Maybe show differences in rank between these two? Some sort of flow-like graph?
sql = """
select 
  counts.id, 
  counts.pubcount,  
  orgs.name,
  orgs.address.country as country,
  orgs.address.latitude as latitude,
  orgs.address.longitude as longitude
from
(
  select 
    orgs.id,
    count(orgs.id) as pubcount
  from (
    SELECT id, orgid
    FROM `covid-19-dimensions-ai.data.publications`,
    UNNEST(research_orgs) orgid
    WHERE type = "article"  -- limit to research articles
    AND ('COVID-19' in UNNEST(concepts.concept) or "new coronavirus" in UNNEST(concepts.concept) or "SARS-CoV-2" in UNNEST(concepts.concept)
          OR 'ncov' in UNNEST(concepts.concept) or "novel coronavirus" in UNNEST(concepts.concept) or "2019-ncov" in UNNEST(concepts.concept)
          OR "severe acute respiratory syndrome coronavirus" in UNNEST(concepts.concept) or 'coronavirus disease 2019' in UNNEST(concepts.concept)
          OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  ) as pubs 
  left join `covid-19-dimensions-ai.data.grid` as orgs on pubs.orgid = orgs.id
  group by orgs.id
) as counts
left join `covid-19-dimensions-ai.data.grid` as orgs on counts.id = orgs.id
order by pubcount DESC

"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 7cf07ede-1545-4694-a6cd-b2ce29c48f60
Job ID: 7cf07ede-1545-4694-a6cd-b2ce29c48f60
Query done.
Cache hit.

Query done.
Cache hit.

Downloading: 100%|███████████████████| 22202/22202 [00:01<00:00, 13149.31rows/s]
Got 22202 rows.

Got 22202 rows.



,id,pubcount,name,country,latitude,longitude
0,grid.38142.3c,3298,Harvard University,United States,42.377052,-71.116653
1,grid.33199.31,1922,Huazhong University of Science and Technology,China,30.508183,114.414742
2,grid.17063.33,1869,University of Toronto,Canada,43.661667,-79.394997
3,grid.21107.35,1860,Johns Hopkins University,United States,39.328888,-76.620277
4,grid.4991.5,1824,University of Oxford,United Kingdom,51.753437,-1.254010
...,...,...,...,...,...,...
22197,grid.490170.b,1,Fuling Center Hospital of Chongqing,China,29.709438,107.395531
22198,grid.459318.2,1,Australian College of Applied Psychology,Australia,-33.865334,151.205322
22199,grid.483908.e,1,Shanghai Hospital Development Center,China,31.238739,121.455162
22200,grid.465956.d,1,LUCA School of Arts,Belgium,50.861362,4.367237


In [43]:
metrics = ["total_times_cited", "total_altmetrics_score", "avg_gield_citation_ratio"]
conditions = ["", "and auth.corresonding = True"]
country = ["", "and 'USA' in auth.raw_affiliation"] # Be sure to add this too—compare local (US) w/ Global leaders
sql = """
WITH top_researchers AS (
  SELECT * FROM (
      SELECT
        auth.researcher_id,
        ANY_VALUE(auth.first_name) as first_name,
        ANY_VALUE(auth.last_name) as last_name,
        COUNT(DISTINCT(pubs.id)) as pub_count,
        SUM(altmetrics.score) as total_altmetrics_score,
        SUM(metrics.times_cited) as total_times_cited,
        AVG(metrics.field_citation_ratio) as avg_field_citation_ratio,
      FROM `covid-19-dimensions-ai.data.publications` as pubs,
      UNNEST(authors) auth
      WHERE type = "article" AND researcher_id is not NULL {}
      AND ('COVID-19' in UNNEST(concepts.concept) or "new coronavirus" in UNNEST(concepts.concept) or "SARS-CoV-2" in UNNEST(concepts.concept)
           OR 'ncov' in UNNEST(concepts.concept) or "novel coronavirus" in UNNEST(concepts.concept) or "2019-ncov" in UNNEST(concepts.concept)
           OR "severe acute respiratory syndrome coronavirus" in UNNEST(concepts.concept) or 'coronavirus disease 2019' in UNNEST(concepts.concept)
           OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
      GROUP BY researcher_id
      ORDER BY {} DESC
    ) 
    WHERE pub_count > 4 -- keep only productive authors, no single-paper wonders
    LIMIT 50
),
researcher_orgs AS (
  -- Get the most frequent affiliation of the researcher
  select researchers.*, orgs.name as org_name, orgs.address.country as org_country, orgs.address.city as org_city
  from (
    SELECT top.researcher_id, org_id, COUNT(DISTINCT(p.id)) as org_count
    FROM `covid-19-dimensions-ai.data.publications` as p
    INNER JOIN top_researchers as top on top.researcher_id in UNNEST(p.authors.researcher_id)
    CROSS JOIN UNNEST(p.authors) auth,
    unnest(auth.grid_ids) as org_id
    WHERE TRUE
    group by top.researcher_id, org_id
    QUALIFY ROW_NUMBER() OVER (PARTITION BY top.researcher_id ORDER BY org_count DESC) = 1
  ) as researchers
  left join `covid-19-dimensions-ai.data.grid` as orgs on orgs.id = researchers.org_id
),
researcher_concepts AS (
  -- Get the concepts associated with papers they published
  SELECT top.researcher_id, ARRAY_CONCAT_AGG(concepts) as concepts
  FROM `covid-19-dimensions-ai.data.publications` as p
  INNER JOIN top_researchers as top on top.researcher_id in UNNEST(p.authors.researcher_id)
  group by top.researcher_id
),
researcher_fields AS (
  -- determines the field of research code in which
  -- the researcher has most frequently authored papers
  SELECT top.researcher_id, for2.name, COUNT(DISTINCT p.id) as cat_count, 
  FROM `covid-19-dimensions-ai.data.publications` as p
  INNER JOIN top_researchers as top on top.researcher_id in UNNEST(p.authors.researcher_id)
  CROSS JOIN UNNEST(category_for.second_level.full) as for2
  WHERE TRUE
  group by top.researcher_id, for2.name
  QUALIFY ROW_NUMBER() OVER (PARTITION BY top.researcher_id ORDER BY cat_count DESC) = 1
)

select 
  top.*,
  fields.* except(cat_count, researcher_id),
  concepts.* except(researcher_id),
  orgs.* except(org_count, researcher_id)
from top_researchers as top
left join researcher_fields as fields on fields.researcher_id = top.researcher_id
left join researcher_concepts as concepts on concepts.researcher_id = top.researcher_id
left join researcher_orgs as orgs on orgs.researcher_id = top.researcher_id

""".format("and auth.corresponding = True", "pub_count")
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

# Rank by each metric for all
# GLobal, corresponidng authors only'
# US-only, all authorships
# US-only, correpsonding authors only

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 2dfcc20a-d9cd-4852-b80f-ee4a53d3c2e3
Job ID: 2dfcc20a-d9cd-4852-b80f-ee4a53d3c2e3
Query done.
Processed: 1.3 GB Billed: 1.3 GB
Query done.
Processed: 1.3 GB Billed: 1.3 GB
Standard price: $0.01 USD

Standard price: $0.01 USD

Downloading: 100%|████████████████████████████| 50/50 [00:06<00:00,  7.49rows/s]
Got 50 rows.

Got 50 rows.

Total time taken 17.77 s.
Finished at 2022-04-27 12:08:21.
Total time taken 17.77 s.
Finished at 2022-04-27 12:08:21.


,researcher_id,first_name,last_name,pub_count,total_altmetrics_score,total_times_cited,avg_field_citation_ratio,name,concepts,org_id,org_name,org_country,org_city
0,ur.0652735016.56,Kwok-Yung,Yuen,66,31091,18262,200.379250,Medical Microbiology,"[{'concept': 'COVID-19 patients', 'relevance':...",grid.194645.b,University of Hong Kong,China,Hong Kong
1,ur.0764344311.13,Nima,Rezaei,52,335,2115,29.191935,Clinical Sciences,"[{'concept': 'serological tests', 'relevance':...",grid.411705.6,Tehran University of Medical Sciences,Iran,Tehran
2,ur.01310657647.41,Giuseppe,Lippi,52,1382,2735,88.101111,Clinical Sciences,"[{'concept': 'laboratory abnormalities', 'rele...",grid.5611.3,University of Verona,Italy,Verona
3,ur.01204711510.82,Alfonso J.,Rodriguez-Morales,50,2186,3625,80.814615,Public Health and Health Services,"[{'concept': 'COVID-19 vaccination', 'relevanc...",grid.441853.f,Fundación Universitaria Autónoma De Las Américas,Colombia,Medellín
4,ur.013163031217.47,Chia Siang,Kow,50,463,190,2.538182,Clinical Sciences,"[{'concept': 'adverse clinical outcomes', 'rel...",grid.411729.8,International Medical University,Malaysia,Kuala Lumpur
5,ur.013145671557.48,Didier,Raoult,39,33340,6141,169.805882,Clinical Sciences,"[{'concept': 'SARS-CoV-2', 'relevance': 0.419}...",grid.483853.1,Méditerranée Infection Foundation,France,Marseille
6,ur.01304412561.27,Daihai,He,38,1108,2983,110.000000,Public Health and Health Services,"[{'concept': 'train transportation', 'relevanc...",grid.16890.36,Hong Kong Polytechnic University,China,Hong Kong
7,ur.01072733407.19,George F.,Gao,38,5606,3128,92.529091,Immunology,"[{'concept': 'COVID-19', 'relevance': 0.397}, ...",grid.458488.d,Institute of Microbiology,China,Beijing
8,ur.014140240160.25,Rujittika,Mungmunpuntipantip,37,18,21,4.990000,Clinical Sciences,"[{'concept': 'COVID-19 vaccine', 'relevance': ...",grid.440681.f,"Dr. D.Y. Patil Vidyapeeth, Pune",India,Pune
9,ur.0716736562.19,Kuldeep,Dhama,37,893,1635,48.973333,Medical Microbiology,"[{'concept': 'risk of infection', 'relevance':...",grid.417990.2,Indian Veterinary Research Institute,India,Bareilly


In [47]:
# Get the top publications
metrics = ["times_cited", "altmetrics_score", "field_citation_ratio"]
sql = """
select
    doi,
    title.preferred as pub_title,
    journal.title as journal_title,
    year, 
    volume,
    issue, 
    pages,
    altmetrics.score as altmetrics_score,
    metrics.times_cited,
    metrics.field_citation_ratio
  from `covid-19-dimensions-ai.data.publications`
  where type = "article"
  AND ('COVID-19' in UNNEST(concepts.concept) or "new coronavirus" in UNNEST(concepts.concept) or "SARS-CoV-2" in UNNEST(concepts.concept)
        OR 'ncov' in UNNEST(concepts.concept) or "novel coronavirus" in UNNEST(concepts.concept) or "2019-ncov" in UNNEST(concepts.concept)
        OR "severe acute respiratory syndrome coronavirus" in UNNEST(concepts.concept) or 'coronavirus disease 2019' in UNNEST(concepts.concept)
        OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  order by {} DESC
  limit 5;
""".format("times_cited")
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

# Rank by each metric for all
# GLobal, corresponidng authors only'
# US-only, all authorships
# US-only, correpsonding authors only

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 8f2c6740-86ac-4b92-b20e-0f5a1fcdfaf2
Job ID: 8f2c6740-86ac-4b92-b20e-0f5a1fcdfaf2
Query done.
Processed: 791.3 MB Billed: 792.0 MB
Query done.
Processed: 791.3 MB Billed: 792.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Downloading: 100%|██████████████████████████████| 5/5 [00:00<00:00, 27.14rows/s]
Got 5 rows.

Got 5 rows.



,doi,pub_title,journal_title,year,volume,issue,pages,altmetrics_score,times_cited,field_citation_ratio
0,10.1016/s0140-6736(20)30183-5,Clinical features of patients infected with 20...,The Lancet,2020,395,10223,497-506,14459,29352,23249.07
1,10.1056/nejmoa2002032,Clinical Characteristics of Coronavirus Diseas...,New England Journal of Medicine,2020,382,18,nejmoa2002032,10119,18648,14770.67
2,10.1056/nejmoa2001017,A Novel Coronavirus from Patients with Pneumon...,New England Journal of Medicine,2020,382,8,727-733,8074,17110,6730.19
3,10.1016/s0140-6736(20)30566-3,Clinical course and risk factors for mortality...,The Lancet,2020,395,10229,1054-1062,13141,16964,10830.36
4,10.1001/jama.2020.1585,Clinical Characteristics of 138 Hospitalized P...,JAMA,2020,323,11,1061-1069,6461,15411,12206.71


In [44]:
# Get the concepts for publications
sql = """
select
    id,
    year, -- get the year so that we can position these over time
    STRING_AGG(c.concept, ',') as concepts -- so we unnest and aggregate, getting rid of the ectra "relevance" tag and other structure info that will add to the download
  from `covid-19-dimensions-ai.data.publications`,
  unnest(concepts) c
  where type = "article"
  and (LOWER(c.concept) IN UNNEST(["covid-19", "new coronavirus", "novel coronavirus", "sars-cov-2", "2019-ncov", "hcov", "hcov-2019", "severe acute respiratory syndrome coronavirus 2", "coronavirus disease 2019"])
    OR REGEXP_CONTAINS(title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  group by id;
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

# Rank by each metric for all
# GLobal, corresponidng authors only'
# US-only, all authorships
# US-only, correpsonding authors only

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 738ace65-3faf-489c-b5fa-0789ed7e64a0
Job ID: 738ace65-3faf-489c-b5fa-0789ed7e64a0
Query done.
Processed: 722.2 MB Billed: 723.0 MB
Query done.
Processed: 722.2 MB Billed: 723.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Downloading: 100%|█████████████████| 245698/245698 [00:06<00:00, 35668.29rows/s]
Got 245698 rows.

Got 245698 rows.

Total time taken 10.58 s.
Finished at 2022-04-27 12:17:31.
Total time taken 10.58 s.
Finished at 2022-04-27 12:17:31.


,id,concepts
0,pub.1137433808,"COVID-19,coronavirus pandemic,pandemic,survey,..."
1,pub.1132948372,"COVID-19,la pandemia COVID-19,urgencias,pandem..."
2,pub.1128243460,"coronavirus disease 2019,disease 2019,communit..."
3,pub.1125649641,"coronavirus disease 2019,disease 2019,sepsis,a..."
4,pub.1135467328,"cardiac magnetic resonance findings,epicardial..."
...,...,...
245693,pub.1130871505,"SARS-CoV-2,severe acute respiratory syndrome c..."
245694,pub.1141113182,"SARS-CoV-2,severe acute respiratory syndrome c..."
245695,pub.1142723603,"SARS-CoV-2,severe acute respiratory syndrome c..."
245696,pub.1143639117,"SARS-CoV-2,severe acute respiratory syndrome c..."


In [52]:
# Get the research organizations 
sql = """
select 
  base.*,
  orgs.name, 
  orgs.types,
  orgs.address.country,
  orgs.address.city,
  orgs_child.name as child_org_name
  from (
    select 
      grant_id,
      grants.funder_org, 
      grants.title,
      grants.funder_org as child_org,
      CASE WHEN hierarchy.parent_id is null THEN grants.funder_org ELSE hierarchy.parent_id END as org_id,
      grants.funding_amount
    from `covid-19-dimensions-ai.data.publications` as pubs
    cross join unnest(supporting_grant_ids) as grant_id
    left join `covid-19-dimensions-ai.data.grants` as grants on grants.id = grant_id
    left join (
      -- get the organizations
      SELECT
        g.id as child_id,
        children.id as parent_id
      FROM
        `covid-19-dimensions-ai.data.grid` g
      CROSS JOIN
        UNNEST(relationships) AS children
      WHERE children.type = "Parent"
    ) as hierarchy on hierarchy.child_id = grants.funder_org
    where pubs.type = "article"
    AND ('COVID-19' in UNNEST(pubs.concepts.concept) or "new coronavirus" in UNNEST(pubs.concepts.concept) or "SARS-CoV-2" in UNNEST(pubs.concepts.concept)
              OR 'ncov' in UNNEST(pubs.concepts.concept) or "novel coronavirus" in UNNEST(pubs.concepts.concept) or "2019-ncov" in UNNEST(pubs.concepts.concept)
              OR "severe acute respiratory syndrome coronavirus" in UNNEST(pubs.concepts.concept) or 'coronavirus disease 2019' in UNNEST(pubs.concepts.concept)
              OR REGEXP_CONTAINS(pubs.title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
    AND funder_org is not Null
) as base 
left join `covid-19-dimensions-ai.data.grid` as orgs on orgs.id = org_id
left join `covid-19-dimensions-ai.data.grid` as orgs_child on orgs_child.id = child_org
where orgs.name is not Null AND funder_org is not Null;
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: 7af37789-e17b-4f9d-921e-f4f84d6f87c7
Job ID: 7af37789-e17b-4f9d-921e-f4f84d6f87c7
Query done.
Processed: 715.0 MB Billed: 715.0 MB
Query done.
Processed: 715.0 MB Billed: 715.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Downloading: 100%|██████████████████████| 2978/2978 [00:00<00:00, 6095.75rows/s]
Got 2978 rows.

Got 2978 rows.



,grant_id,funder_org,title,child_org,org_id,funding_amount,name,types,country,city,child_org_name
0,grant.9343181,grid.426413.6,When pandemic and everyday ethics collide: sup...,grid.426413.6,grid.496779.2,258812.0,UK Research and Innovation,[Government],United Kingdom,Swindon,Arts and Humanities Research Council
1,grant.9845960,grid.281076.a,Leveraging community health workers to improve...,grid.281076.a,grid.94365.3d,751585.0,National Institutes of Health,[Government],United States,Bethesda,National Institute on Minority Health and Heal...
2,grant.9846032,grid.416868.5,Sex specific immune response to SARS-CoV-2 lea...,grid.416868.5,grid.94365.3d,766078.0,National Institutes of Health,[Government],United States,Bethesda,National Institute of Mental Health
3,grant.9453205,grid.453115.7,Development of Nanopore Sequencing-based Platf...,grid.453115.7,grid.453115.7,1860470.0,Innovation and Technology Commission,[Government],China,Hong Kong,Innovation and Technology Commission
4,grant.9959631,grid.473834.f,Measurement of the Aerosol Generating Potentia...,grid.473834.f,grid.451056.3,399470.0,National Institute for Health Research,[Government],United Kingdom,London,NIHR Academy
...,...,...,...,...,...,...,...,...,...,...,...
2973,grant.9535866,grid.420089.7,Identifying biomarker signatures of prognostic...,grid.420089.7,grid.94365.3d,1685063.0,National Institutes of Health,[Government],United States,Bethesda,Eunice Kennedy Shriver National Institute of C...
2974,grant.9535863,grid.420089.7,AICORE-kids: Artificial Intelligence COVID-19 ...,grid.420089.7,grid.94365.3d,1595976.0,National Institutes of Health,[Government],United States,Bethesda,Eunice Kennedy Shriver National Institute of C...
2975,grant.9535864,grid.420089.7,Diagnosis of MIS-C in febrile children,grid.420089.7,grid.94365.3d,1750391.0,National Institutes of Health,[Government],United States,Bethesda,Eunice Kennedy Shriver National Institute of C...
2976,grant.9535863,grid.420089.7,AICORE-kids: Artificial Intelligence COVID-19 ...,grid.420089.7,grid.94365.3d,1595976.0,National Institutes of Health,[Government],United States,Bethesda,Eunice Kennedy Shriver National Institute of C...


In [55]:
# Get the publications over time & data that I'm interested in working with
sql = """
select 
  ARRAY_CONCAT_AGG(pubs.research_org_countries) as countries,
  ARRAY_CONCAT_AGG(pubs.research_org_state_codes) as states,
  ANY_VALUE(DATE_TRUNC(SAFE_CAST(pubs.date_online AS DATE), MONTH)) as published_date,
  STRING_AGG(c.name, ",") as fields,
  ANY_VALUE(journal.title) as journal_title, -- the title of the journal
  ANY_VALUE(metrics.times_cited) as times_cited,
  ANY_VALUE(altmetrics.score) as altmetrics_score
  from `covid-19-dimensions-ai.data.publications` as pubs, 
  unnest(category_for.second_level.full) c
  where pubs.type = "article"
  AND ('COVID-19' in UNNEST(pubs.concepts.concept) or "new coronavirus" in UNNEST(pubs.concepts.concept) or "SARS-CoV-2" in UNNEST(pubs.concepts.concept)
       OR 'ncov' in UNNEST(pubs.concepts.concept) or "novel coronavirus" in UNNEST(pubs.concepts.concept) or "2019-ncov" in UNNEST(pubs.concepts.concept)
       OR "severe acute respiratory syndrome coronavirus" in UNNEST(pubs.concepts.concept) or 'coronavirus disease 2019' in UNNEST(pubs.concepts.concept)
       OR REGEXP_CONTAINS(pubs.title.preferred, r"covid-19|new coronavirus|novel coronavirus|sars-cov-2|2019-ncov|hcov|severe acute respiratory syndrome coronavirus|coronavirus disease 2019"))
  group by (pubs.id)
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)
df

Requesting query... 
Requesting query... 
Query running...
Query running...
Job ID: c6d75dfb-c52c-484e-9e25-1f3d67e2c62f
Job ID: c6d75dfb-c52c-484e-9e25-1f3d67e2c62f
Query done.
Processed: 799.9 MB Billed: 800.0 MB
Query done.
Processed: 799.9 MB Billed: 800.0 MB
Standard price: $0.00 USD

Standard price: $0.00 USD

Downloading: 100%|█████████████████| 213197/213197 [00:18<00:00, 11740.13rows/s]
Got 213197 rows.

Got 213197 rows.

Total time taken 24.9 s.
Finished at 2022-04-27 20:35:16.
Total time taken 24.9 s.
Finished at 2022-04-27 20:35:16.


,countries,states,published_date,fields,journal_title,times_cited,altmetrics_score
0,"[US, US]","[US-CO, US-CO]",2022-12-01,"Statistics,Public Health and Health Services",Epidemiology,0,6
1,[FR],[],2020-02-01,Clinical Sciences,Plastic & Reconstructive Surgery,2,<NA>
2,[US],[US-MA],2020-01-01,Clinical Sciences,New England Journal of Medicine,4202,10149
3,[],[],2020-02-01,Clinical Sciences,Chinese Journal of Otolaryngology Head and Nec...,3,<NA>
4,"[CN, CN]",[],2020-02-01,"Clinical Sciences,Public Health and Health Ser...",Chinese Journal of Pediatrics,88,553
...,...,...,...,...,...,...,...
213192,[AU],[AU-VIC],2021-03-01,Public Health and Health Services,Nutrients,1,1
213193,"[DK, NO]",[],2021-03-01,Public Health and Health Services,Scandinavian Journal of Gastroenterology,2,<NA>
213194,[CN],[],2021-03-01,Psychology,Psychology Health & Medicine,5,<NA>
213195,"[US, IT]","[US-GA, US-NY]",2021-03-01,Public Health and Health Services,Journal of Health Communication,11,23
